In [1]:
import os
import pandas as pd
import numpy as np
import glob
from pathlib import Path

col13 = pd.read_csv('/home/kchen/Documents/leak/data/puf_tar_col_2013.csv', index_col='CASEID', low_memory=False)
puf13 = pd.read_csv('/home/kchen/Documents/leak/data/acs_nsqip_puf13.csv', index_col='CaseID', low_memory=False)

col13.columns = map(str.upper, col13.columns)
puf13.columns = map(str.upper, puf13.columns)

In [2]:
proc = pd.read_csv(r'/home/kchen/Documents/nsqip_raw/pro/combined/pro.csv', index_col='CASEID', low_memory=False)


In [3]:
colec = pd.read_csv(r'/home/kchen/Documents/nsqip_raw/col/combined/col.csv', index_col='CASEID', low_memory=False)

In [4]:
procols = [column for column in proc.columns if column.startswith('PRO')]


In [5]:
procols

['PRO_TUMORLOC',
 'PRO_CHEMO',
 'PRO_CHEMO_UNK',
 'PRO_RADIO',
 'PRO_RADIO_UNK',
 'PRO_PRESTAGE_T',
 'PRO_PRELYMPH_N',
 'PRO_PREDISTANTM_M',
 'PRO_COLON',
 'PRO_STOMA',
 'PRO_PATHSTAGE_T',
 'PRO_PATHLYMPH_N',
 'PRO_NUMNODES',
 'PRO_NUMNODES_UNK',
 'PRO_NUMNODES_NA',
 'PRO_PATHDISTANTM_M',
 'PRO_MARG_RADIAL',
 'PRO_CLEAR_RADIAL',
 'PRO_CLEAR_RADIAL_UNK',
 'PRO_MARG_DISTAL',
 'PRO_MARG_DISTAL_UNK',
 'PRO_CLEAR_DISTAL',
 'PRO_CLEAR_DISTAL_UNK',
 'PRO_APPROACH',
 'PRO_ANASTOMIC',
 'PRO_ILEUS',
 'PRO_ILEUS_UNK']

In [6]:
colcols = [column for column in colec.columns if column.startswith('COL')]


In [7]:
for col in procols:
    print(col, proc[col].isnull().sum()/len(proc))

PRO_TUMORLOC 0.503903051476609
PRO_CHEMO 0.00403952180795895
PRO_CHEMO_UNK 0.0
PRO_RADIO 0.0048583437960587365
PRO_RADIO_UNK 0.0
PRO_PRESTAGE_T 0.5234456029259239
PRO_PRELYMPH_N 0.5225176046727441
PRO_PREDISTANTM_M 0.5759047982968503
PRO_COLON 0.0
PRO_STOMA 0.0
PRO_PATHSTAGE_T 0.5192969048528849
PRO_PATHLYMPH_N 0.5195152573830449
PRO_NUMNODES 0.0
PRO_NUMNODES_UNK 0.0
PRO_NUMNODES_NA 0.0
PRO_PATHDISTANTM_M 0.5933730007096457
PRO_MARG_RADIAL 0.5302145313608821
PRO_CLEAR_RADIAL 0.0
PRO_CLEAR_RADIAL_UNK 0.0
PRO_MARG_DISTAL 0.5326709973251815
PRO_MARG_DISTAL_UNK 0.0
PRO_CLEAR_DISTAL 0.0
PRO_CLEAR_DISTAL_UNK 0.0
PRO_APPROACH 0.0
PRO_ANASTOMIC 0.0
PRO_ILEUS 0.0003275287952399148
PRO_ILEUS_UNK 0.0


In [8]:
proc['PRO_TUMORLOC'].value_counts(dropna=False)

NaN                                       9231
Lower third (<5 cm from anal verge)       3956
Middle third (5-10 cm from anal verge)    2429
Unknown                                   1422
Upper third (>10 cm from anal verge)      1281
Name: PRO_TUMORLOC, dtype: int64

In [9]:
proc['PRO_TUMORLOC'] = proc['PRO_TUMORLOC'].fillna('Unknown')
proc['PRO_TUMORLOC'].value_counts(dropna=False)

Unknown                                   10653
Lower third (<5 cm from anal verge)        3956
Middle third (5-10 cm from anal verge)     2429
Upper third (>10 cm from anal verge)       1281
Name: PRO_TUMORLOC, dtype: int64

In [10]:
proc.loc[proc['PRO_TUMORLOC'] != 'Unknown', 'COL_INDICATION'] = 'Rectal cancer'
proc['COL_INDICATION'].value_counts(dropna=False)

NaN              10653
Rectal cancer     7666
Name: COL_INDICATION, dtype: int64

In [11]:
proc['COL_INDICATION'].fillna(value='Unknown', inplace=True)
proc['COL_INDICATION'].value_counts(dropna=False)

Unknown          10653
Rectal cancer     7666
Name: COL_INDICATION, dtype: int64

In [12]:
colec['COL_INDICATION'].value_counts()

Colon cancer                         90518
Chronic diverticular disease         36189
Other-Enter ICD-10 for diagnosis     30063
Acute diverticulitis                 21806
Non-malignant polyp                  20128
Other-Enter ICD-9 for diagnosis      16393
Crohn's Disease                      14805
Colon cancer w/ obstruction          11920
Volvulus                              7141
Ulcerative colitis                    5322
Bleeding                              1795
Enterocolitis (e.g. C. Difficile)     1471
Unknown                                361
Name: COL_INDICATION, dtype: int64

In [13]:
proc['COL_INDICATION'] = 'Unknown'
proc['COL_INDICATION'].value_counts()

Unknown    18319
Name: COL_INDICATION, dtype: int64

In [14]:
proc.loc[(proc['PODIAGTX'].str.contains('rect', case=False, na=False) & proc['PODIAGTX'].str.contains('malig', case=False, na=False)), 'COL_INDICATION'] = 'Rectal cancer'
proc.loc[(proc['PODIAGTX'].str.contains('rect', case=False, na=False) & proc['PODIAGTX'].str.contains('carc', case=False, na=False)), 'COL_INDICATION'] = 'Rectal cancer'
proc.loc[(proc['PODIAGTX'].str.contains('ana', case=False, na=False) & proc['PODIAGTX'].str.contains('malig', case=False, na=False)), 'COL_INDICATION'] = 'Anal cancer'
proc.loc[proc['PODIAGTX'].str.contains('ulcer', case=False, na=False), 'COL_INDICATION'] = 'Ulcerative colitis'
proc.loc[proc['PODIAGTX'].str.contains('croh', case=False, na=False), 'COL_INDICATION'] = 'Crohn\'s disease'
proc.loc[proc['PODIAGTX'].str.contains('prola', case=False, na=False), 'COL_INDICATION'] = 'Rectal prolapse'
proc['COL_INDICATION'].value_counts()

Unknown               18066
Rectal cancer           169
Ulcerative colitis       46
Rectal prolapse          35
Anal cancer               3
Name: COL_INDICATION, dtype: int64

In [15]:
proc.loc[(proc['PODIAGTX10'].str.contains('rec', case=False, na=False) & proc['PODIAGTX10'].str.contains('mal', case=False, na=False)), 'COL_INDICATION'] = 'Rectal cancer'
proc.loc[(proc['PODIAGTX10'].str.contains('rect', case=False, na=False) & proc['PODIAGTX10'].str.contains('carc', case=False, na=False)), 'COL_INDICATION'] = 'Rectal cancer'
proc.loc[(proc['PODIAGTX10'].str.contains('rec', case=False, na=False) & proc['PODIAGTX10'].str.contains('neo', case=False, na=False)), 'COL_INDICATION'] = 'Rectal cancer'
proc.loc[(proc['PODIAGTX10'].str.contains('col', case=False, na=False) & proc['PODIAGTX10'].str.contains('mal', case=False, na=False)), 'COL_INDICATION'] = 'Rectal cancer'
proc.loc[(proc['PODIAGTX10'].str.contains('col', case=False, na=False) & proc['PODIAGTX10'].str.contains('neo', case=False, na=False)), 'COL_INDICATION'] = 'Rectal cancer'
proc.loc[(proc['PODIAGTX10'].str.contains('ana', case=False, na=False) & proc['PODIAGTX10'].str.contains('malig', case=False, na=False)), 'COL_INDICATION'] = 'Anal cancer'
proc.loc[proc['PODIAGTX10'].str.contains('ulcer', case=False, na=False), 'COL_INDICATION'] = 'Ulcerative colitis'
proc.loc[proc['PODIAGTX10'].str.contains('cro', case=False, na=False), 'COL_INDICATION'] = 'Crohn\'s Disease'
proc.loc[proc['PODIAGTX10'].str.contains('prola', case=False, na=False), 'COL_INDICATION'] = 'Rectal prolapse'
proc['COL_INDICATION'].value_counts()

Rectal cancer         8392
Ulcerative colitis    3366
Unknown               2481
Rectal prolapse       2472
Crohn's Disease       1221
Anal cancer            387
Name: COL_INDICATION, dtype: int64

In [16]:
proc.loc[proc['PODIAGTX'] == 'MALIGNANT NEOPLASM OF RECTUM', 'COL_INDICATION'] = 'Rectal cancer'
proc.loc[proc['PODIAGTX'] == 'MALIGNANT NEOPLASM OF RECTOSIGMOID JUNCTION', 'COL_INDICATION'] = 'Rectal cancer'
proc.loc[proc['PODIAGTX'] == 'MALIGNANT NEOPLASM OF RECTUM RECTOSIGMOID JUNCTION AND ANUS', 'COL_INDICATION'] = 'Rectal cancer'
proc.loc[proc['PODIAGTX'] == 'MALIGNANT NEOPLASM OF COLON', 'COL_INDICATION'] = 'Rectal cancer'
proc.loc[proc['PODIAGTX'] == 'MALIGNANT NEOPLASM OF SIGMOID COLON', 'COL_INDICATION'] = 'Rectal cancer'
proc.loc[proc['PODIAGTX'] == 'CARCINOMA IN SITU OF RECTUM', 'COL_INDICATION'] = 'Rectal cancer'
proc.loc[proc['PODIAGTX'] == 'NEOPLASM OF UNCERTAIN BEHAVIOR OF STOMACH INTESTINES AND RECTUM', 'COL_INDICATION'] = 'Rectal cancer'

proc.loc[proc['PODIAGTX'] == 'ULCERATIVE COLITIS UNSPECIFIED', 'COL_INDICATION'] = 'Ulcerative colitis'
proc.loc[proc['PODIAGTX'] == 'ULCERATIVE (CHRONIC) PROCTITIS', 'COL_INDICATION'] = 'Ulcerative colitis'
proc.loc[proc['PODIAGTX'] == 'ULCERATIVE (CHRONIC) ENTEROCOLITIS', 'COL_INDICATION'] = 'Ulcerative colitis'
proc.loc[proc['PODIAGTX'] == 'ULCERATIVE (CHRONIC) PROCTOSIGMOIDITIS', 'COL_INDICATION'] = 'Ulcerative colitis'
proc.loc[proc['PODIAGTX'] == 'LEFT-SIDED ULCERATIVE (CHRONIC) COLITIS', 'COL_INDICATION'] = 'Ulcerative colitis'

In [17]:
pd.DataFrame(proc['PODIAGTX10'].value_counts(dropna=False)).iloc[0:30,:]

,PODIAGTX10
Malignant neoplasm of rectum,5159
Rectal prolapse,1887
MALIGNANT NEOPLASM OF RECTUM,1397
"Ulcerative colitis, unspecified, without complications",905
RECTAL PROLAPSE,533
Malignant neoplasm of rectosigmoid junction,450
ULCERATIVE COLITIS UNSPECIFIED WITHOUT COMP,347
NaN,339
Ulcerative (chronic) pancolitis without complications,218
"Malignant neoplasm of anus, unspecified",198


In [18]:
proc.loc[proc['PODIAGTX10'] == 'Malignant neoplasm of rectum', 'COL_INDICATION'] = 'Rectal cancer'
proc.loc[proc['PODIAGTX10'] == 'MALIGNANT NEOPLASM OF RECTUM', 'COL_INDICATION'] = 'Rectal cancer'
proc.loc[proc['PODIAGTX10'] == 'Malignant neoplasm of rectosigmoid junction', 'COL_INDICATION'] = 'Rectal cancer'
proc.loc[proc['PODIAGTX10'] == 'MALIGNANT NEOPLASM OF RECTOSIGMOID JUNCTION', 'COL_INDICATION'] = 'Rectal cancer'
proc.loc[proc['PODIAGTX10'] == 'MALIGNANT NEOPLASM OF RECTUM RECTOSIGMOID JUNCTION AND ANUS', 'COL_INDICATION'] = 'Rectal cancer'
proc.loc[proc['PODIAGTX10'] == 'MALIGNANT NEOPLASM OF COLON', 'COL_INDICATION'] = 'Rectal cancer'
proc.loc[proc['PODIAGTX10'] == 'MALIGNANT NEOPLASM OF SIGMOID COLON', 'COL_INDICATION'] = 'Rectal cancer'
proc.loc[proc['PODIAGTX10'] == 'CARCINOMA IN SITU OF RECTUM', 'COL_INDICATION'] = 'Rectal cancer'
proc.loc[proc['PODIAGTX10'] == 'Malignant neoplasm of sigmoid colon', 'COL_INDICATION'] = 'Rectal cancer'
proc.loc[proc['PODIAGTX10'] == 'Malignant neoplasm of overlapping sites of rectum, anus and anal canal', 'COL_INDICATION'] = 'Rectal cancer'
proc.loc[proc['PODIAGTX10'] == 'Malignant neoplasm of colon, unspecified', 'COL_INDICATION'] = 'Rectal cancer'


proc.loc[proc['PODIAGTX10'] == 'Rectal prolapse', 'COL_INDICATION'] = 'Rectal prolapse'
proc.loc[proc['PODIAGTX10'] == 'RECTAL PROLAPSE', 'COL_INDICATION'] = 'Rectal prolapse'

proc.loc[proc['PODIAGTX10'] == 'Malignant neoplasm of anus, unspecified', 'COL_INDICATION'] = 'Anal cancer'
proc.loc[proc['PODIAGTX10'] == 'Malignant neoplasm of anal canal', 'COL_INDICATION'] = 'Anal cancer'

proc.loc[proc['PODIAGTX10'] == 'Crohn\'s disease of large intestine without complications', 'COL_INDICATION'] = 'Crohn\'s Disease'
proc.loc[proc['PODIAGTX10'] == 'Crohn\'s disease of large intestine with fistula', 'COL_INDICATION'] = 'Crohn\'s Disease'
proc.loc[proc['PODIAGTX10'] == 'Crohn\'s disease of large intestine with other complication', 'COL_INDICATION'] = 'Crohn\'s Disease'
proc.loc[proc['PODIAGTX10'] == 'Crohn\'s disease, unspecified, without complications', 'COL_INDICATION'] = 'Crohn\'s Disease'


proc.loc[proc['PODIAGTX10'] == 'ULCERATIVE COLITIS UNSPECIFIED WITHOUT COMP', 'COL_INDICATION'] = 'Ulcerative colitis'
proc.loc[proc['PODIAGTX10'] == 'Ulcerative (chronic) pancolitis without complications', 'COL_INDICATION'] = 'Ulcerative colitis'
proc.loc[proc['PODIAGTX10'] == 'Ulcerative (chronic) proctitis without complications', 'COL_INDICATION'] = 'Ulcerative colitis'
proc.loc[proc['PODIAGTX10'] == 'Ulcerative colitis, unspecified with unspecified complications', 'COL_INDICATION'] = 'Ulcerative colitis'
proc.loc[proc['PODIAGTX10'] == 'Ulcerative (chronic) pancolitis with rectal bleeding', 'COL_INDICATION'] = 'Ulcerative colitis'
proc.loc[proc['PODIAGTX10'] == 'Ulcerative (chronic) pancolitis with other complication	', 'COL_INDICATION'] = 'Ulcerative colitis'
proc.loc[proc['PODIAGTX10'] == 'ULCERATIVE COLITIS UNSPECIFIED', 'COL_INDICATION'] = 'Ulcerative colitis'
proc.loc[proc['PODIAGTX10'] == 'ULCERATIVE (CHRONIC) PROCTITIS', 'COL_INDICATION'] = 'Ulcerative colitis'
proc.loc[proc['PODIAGTX10'] == 'ULCERATIVE (CHRONIC) ENTEROCOLITIS', 'COL_INDICATION'] = 'Ulcerative colitis'
proc.loc[proc['PODIAGTX10'] == 'ULCERATIVE (CHRONIC) PROCTOSIGMOIDITIS', 'COL_INDICATION'] = 'Ulcerative colitis'
proc.loc[proc['PODIAGTX10'] == 'LEFT-SIDED ULCERATIVE (CHRONIC) COLITIS', 'COL_INDICATION'] = 'Ulcerative colitis'
proc.loc[proc['PODIAGTX10'] == 'Ulcerative colitis, unspecified with rectal bleeding', 'COL_INDICATION'] = 'Ulcerative colitis'
proc.loc[proc['PODIAGTX10'] == 'Ulcerative colitis', 'COL_INDICATION'] = 'Ulcerative colitis'

In [19]:
proc['COL_INDICATION'].value_counts()

Rectal cancer         8571
Ulcerative colitis    3366
Rectal prolapse       2472
Unknown               2269
Crohn's Disease       1221
Anal cancer            420
Name: COL_INDICATION, dtype: int64

In [20]:
columns = {'PRO_CHEMO':'COL_CHEMO', 'PRO_NUMNODES':'COL_NODESEVAL', 'PRO_APPROACH':'COL_APPROACH', 'PRO_ANASTOMIC':'COL_ANASTOMOTIC', 'PRO_ILEUS':'COL_ILEUS', 'PRO_PATHSTAGE_T':'COL_MALIGNANCYT','PRO_PATHLYMPH_N':'COL_MALIGNANCYN','PRO_PATHDISTANTM_M':'COL_MALIGNANCYM','PRO_STOMA':'COL_STOMA'}
proc.rename(columns=columns, inplace=True, errors="raise")

In [21]:
proc['COL_STEROID'] = 'Unknown'
proc['COL_ORAL_ANTIBIOTIC'] = 'Unknown'
proc['COL_MECH_BOWEL_PREP'] = 'Unknown'
proc['COL_EMERGENT'] = 'Unknown'


In [22]:
procols2 = [column for column in proc.columns if column.startswith('PRO')]

In [23]:
proc.drop(procols2, axis=1, inplace=True)

In [24]:
print(proc.shape, colec.shape)

(18319, 289) (257913, 352)


In [25]:
join = pd.concat([proc, colec])
join.shape

(276232, 353)

In [26]:
join.to_csv(r'/home/kchen/Documents/nsqip_raw/procol/procol.csv')

In [27]:
join.reset_index(inplace=True)

<ipython-input-27-c3a1c367a89b>:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  join.reset_index(inplace=True)


In [28]:
import feather
join.to_feather(r'/home/kchen/Documents/nsqip_raw/procol/procol.feather')

ArrowInvalid: ("Could not convert '49204' with type str: tried to convert to double", 'Conversion failed for column OTHERCPT4 with type object')